<a href="https://colab.research.google.com/github/Ayushhsingh005/Ai_Travel_Agent/blob/main/Copy_of_Ai_travel_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import os
from openai import OpenAI
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

client = OpenAI()


# TOOL 1: Get REAL destination data from Wikipedia
def get_destination_info(location):
    if not location or not isinstance(location, str):
        return "Invalid destination provided."

    prompt = f"""
You are a travel expert.

Give EXACTLY 2 SHORT LINES about the destination below.

Destination: {location}

Format strictly:
Line 1: why that location is famous
Line 2: Top attraction + best time to visit
Line 3: 2-3 best location of there


Rules:
- Do NOT exceed 2 lines
- Keep each line under 15 words
- Do NOT add extra text
- If destination does not exist, say so in one short line
"""

    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.1
        )

        return response.choices[0].message.content.strip()

    except Exception as e:
        return f"Error fetching destination information: {str(e)}"

# TOOL2: Budget calculate
def calculate_budget(total_budget, days):
    return {
        "Travel (one time)": int(total_budget * 0.30),
        "Hotel (per day)": int((total_budget * 0.40) / days),
        "Food (per day)": int((total_budget * 0.20) / days),
        "Local Travel (per day)": int((total_budget * 0.10) / days)
    }

# TOOL3: Day Wise Plan
def generate_itinerary(location, days):
    if not location or days <= 0:
        return "Invalid destination or number of days."

    prompt = f"""
You are a professional travel planner.

Create a short, concise day-wise itinerary for {location} for {days} days.

Rules:
- Exactly ONE line per day
- Each line must be SHORT and PRECISE
- No extra explanations
- Output format strictly:
  Day 1: ...
  Day 2: ...
  ...
"""

    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.2
        )

        return response.choices[0].message.content.strip()

    except Exception as e:
        return f"Error generating itinerary: {str(e)}"

SecretNotFoundError: Secret OPENAI_API_KEY does not exist.

In [ ]:
class TravelPlanningAgent:
    def __init__(self, location, days, budget):
        self.location = location
        self.days = days
        self.budget = budget

    def execute(self):

        destination_info = get_destination_info(self.location)
        budget_plan = calculate_budget(self.budget, self.days)
        itinerary = generate_itinerary(self.location, self.days)

        travel_plan = f"""

AI TRAVEL PLANNING AGENT

Destination: {self.location}
Duration: {self.days} days
Total Budget: ₹{self.budget}

ABOUT DESTINATION (REAL DATA):
{destination_info}

BUDGET BREAKDOWN:
- Travel Cost: ₹{budget_plan['Travel (one time)']}
- Hotel Per Day: ₹{budget_plan['Hotel (per day)']}
- Food Per Day: ₹{budget_plan['Food (per day)']}
- Local Travel Per Day: ₹{budget_plan['Local Travel (per day)']}

DAY-WISE ITINERARY:
{itinerary}

"""
        return travel_plan

In [ ]:
location = input("Enter travel destination (e.g., Delhi): ")
days = int(input("Enter number of days: "))
budget_input = input("Enter total budget (INR): ")
budget = int(budget_input.replace(',', ''))

agent = TravelPlanningAgent(location, days, budget)
result = agent.execute()

print(result)